# Consultas em SQL no Big Query - registro de código SQL

In [ ]:
SELECT date,
       store_name,
       category_name,
       count(distinct invoice_and_item_number) as transactions,
       sum(volume_sold_gallons) as total_volume_sold_gallons,
       sum(volume_sold_liters) as total_volume_sold_liters,
       sum(sale_dollars) as total_revenue_sale,
       avg(sale_dollars) as average_revenue_sale,
       min(sale_dollars) as min_revenue_sale,
       max(sale_dollars) as max_revenue_sale
FROM `bigquery-public-data.iowa_liquor_sales.sales`
GROUP BY date,
       store_name,
       category_name
ORDER BY date

In [ ]:
#barra e asterisco pra comentar campos
SELECT date,
       /*store_name,
       category_name,*/
       count(distinct invoice_and_item_number) as transactions,
       sum(volume_sold_gallons) as total_volume_sold_gallons,
       sum(volume_sold_liters) as total_volume_sold_liters,
       sum(sale_dollars) as total_revenue_sale,
       avg(sale_dollars) as average_revenue_sale,
       min(sale_dollars) as min_revenue_sale,
       max(sale_dollars) as max_revenue_sale
FROM `bigquery-public-data.iowa_liquor_sales.sales`
GROUP BY date
       /*store_name,
       category_name*/
ORDER BY date

Para exportar os dados manipulados no BigQuery há diversas opções: CSV, JSON, Planilhas Google, etc.
Curiosidade: dá pra "organizar" um csv em excel com a função text to columns, pedindo o delimitador

# Integração com BigQuery

In [ ]:
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

project = 'manifest-shell-398219'
location = 'US'
client = bigquery.Client(project = project, location = location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

In [ ]:
#pega o job do bigquery (esses dados estão em info do job)
job = client.get_job('bquxjob_24176979_18a806dcbdd')
#transforma esse job em dataframe
results = job.to_dataframe()
results

,date,pack,total_sale_dollars,total_transactions
0,2012-01-03,123573,1012493.81,10147
1,2012-01-04,111053,860053.73,9263
2,2012-01-05,114268,940194.93,8795
3,2012-01-09,109871,933835.50,9186
4,2012-01-10,106619,900077.61,8732
...,...,...,...,...
2993,2023-08-27,12,42.00,1
2994,2023-08-28,60876,1369530.95,5344
2995,2023-08-29,172704,2421791.18,14438
2996,2023-08-30,95889,1797491.44,8725


Outra forma de fazer isso é pelo BigQuery. No botão exportar dados, tem todo esse código que fizemos acima de forma automática.

# Criar modelo de ML de clustering (kmeans) no BigQuery

In [ ]:
#criar modelo:
CREATE OR REPLACE MODEL `manifest-shell-398219.model_clusters.store_retail_cluster_model` OPTIONS (model_type = 'kmeans', num_clusters = 6) AS
SELECT store_name,
       count(distinct invoice_and_item_number) as transactions,
       sum(pack) as pack,
       sum(state_bottle_cost) as state_bottle_cost,
       sum(sale_dollars) as sale_dollars,
       sum(volume_sold_gallons) as volume_sold_gallons,
       sum(volume_sold_liters) as volume_sold_liters
FROM `bigquery-public-data.iowa_liquor_sales.sales`
GROUP BY store_name
ORDER BY store_name

In [ ]:
#fazer consulta no modelo
SELECT count(distinct store_name) as store_total, centroid_id FROM ML.PREDICT(MODEL `manifest-shell-398219.model_clusters.store_retail_cluster_model`,
(SELECT store_name,
       count(distinct invoice_and_item_number) as transactions,
       sum(pack) as pack,
       sum(state_bottle_cost) as state_bottle_cost,
       sum(sale_dollars) as sale_dollars,
       sum(volume_sold_gallons) as volume_sold_gallons,
       sum(volume_sold_liters) as volume_sold_liters
FROM `bigquery-public-data.iowa_liquor_sales.sales`
GROUP BY store_name
ORDER BY store_name))
GROUP BY centroid_id
ORDER BY centroid_id